Ferramentas de NLP usando o dataset de tweets

Modelo KNN

Pré Processamento

70/30 Treinamento


In [1]:
from google.colab import drive
from collections import Counter
import pandas as pd
import nltk
import re
from sklearn.neighbors import KNeighborsClassifier
from nltk.tokenize.casual import casual_tokenize
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import numpy as np

Dando permissão para o colab poder acessar os arquivos do meu drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
nltk.download('stopwords')
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [136]:
df_train = pd.read_csv('/content/drive/My Drive/NLP/DataSet da Primeira Competição/train.csv')
df_test = pd.read_csv('/content/drive/My Drive/NLP/DataSet da Primeira Competição/test.csv')


# df.head()

X_df_test = df_test.Text.values
y_df_test = np.zeros([1640,])

X_df_train = df_train.Text.values
y_df_train = df_train.Classificacao.values


X_df_train = array(X_df_train)
X_df_test = array((X_df_test))
y_df_train = array(y_df_train)
y_df_test = array(y_df_test)

X = np.append(X_df_train,X_df_test)
y = np.append(y_df_train,y_df_test)

# y = pd.get_dummies(y)

Para Testar a acuracia no meu conjunto de treino.

In [148]:
X = df_train.Text
y = df_train.Classificacao

In [ ]:
X.shape

(8199,)

In [149]:
stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')
stop_words_en = nltk.corpus.stopwords.words('english')
stop_words_spn = nltk.corpus.stopwords.words('spanish')

stop_words = stop_words_ptbr + stop_words_en + stop_words_spn

bags_of_words = []
vocabulary = []



for text in X:
  sentence = text
  # x = re.sub(r'http\S+', '', text)
  tokens = casual_tokenize(sentence)
  # tokens = [word for word in tokens if word.isalnum()]
  # puncs = set((',', '.', '--','-','!','?',':', ';', '``', "''", '(', ')', '[', ']','\u2026','"'))
  # tokens = (x.lower() for x in tokens if x not in puncs)
  # tokens = Counter(tokens)
  tokens = [x2 for x2 in tokens if x2 not in stop_words]
  # vocabulary += [" ".join([x for x in tokens])]
  bags_of_words.append(Counter(tokens))


# count_vect = CountVectorizer()
# df_bows = count_vect.fit_transform(vocabulary)

df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)

# bow = []
# from collections import Counter
# for text in X:
#   sentence = text

#   tokens = casual_tokenize(sentence) 
#   tokens = [x for x in tokens if x not in stop_words]

#   bow.append(Counter(tokens))
#   # vocabulary += [" ".join([x for x in tokens])]

# bows = pd.DataFrame.from_records(bow)
# bows = bows.fillna(0).astype(int)


df_bows.shape


(6559, 12405)

In [150]:
stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')
stop_words_en = nltk.corpus.stopwords.words('english')
stop_words_spn = nltk.corpus.stopwords.words('spanish')

stop_words = stop_words_ptbr + stop_words_en + stop_words_spn

vocabulary = []
bow = []

for text in X:
  sentence = text

  tokens = casual_tokenize(sentence) # strip_handles=True remove @user
  tokens = [x for x in tokens if x not in stop_words]
  vocabulary += [" ".join([x for x in tokens])]

count_vect = CountVectorizer()
df_bows = count_vect.fit_transform(vocabulary)

df_bows.shape

(6559, 11143)

In [151]:
X_train, X_test, y_train, y_test = train_test_split(df_bows,y, test_size = 0.2, random_state = 109,shuffle=False)

(6559, 13030)

In [155]:
# clf = svm.SVC(decision_function_shape="ovr", kernel= "linear")

# parameters = {'loss': ('log','hinge','perceptron','modified_huber','huber')} #hinge é o melhor

# clf = SGDClassifier(loss='hinge',max_iter=10000, shuffle=False,penalty='elasticnet',n_iter_no_change=10) #o melhor até agora
# clf = SGDClassifier(loss='hinge',max_iter=10000, shuffle=False,n_iter_no_change=10)
# clf = SGDClassifier(loss='hinge',max_iter=10000, penalty='elasticnet', shuffle=False) 
# clf = SGDClassifier(loss='hinge',shuffle=False)

clf = svm.SVC(decision_function_shape="ovr", kernel= "linear")

clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [156]:
predict = clf.predict(X_test)

In [141]:
final = pd.DataFrame(df_test.Id)
final['Category'] = predict

final.to_csv('/content/drive/MyDrive/NLP/DataSet da Primeira Competição/finalSGD22.csv',index=False)

# df.to_csv('/content/drive/MyDrive/NLP/DataSet da Primeira Competição/final.csv', index=False)

In [157]:
from sklearn.metrics import accuracy_score as acs

print(acs(predict, y_test))

0.9626524390243902


In [ ]:
predict.shape

(1640,)

In [ ]:
df_test.Id

0       3568
1       1323
2       7976
3       2408
4       4435
        ... 
1635    3536
1636    6881
1637     627
1638    2165
1639    4268
Name: Id, Length: 1640, dtype: int64

In [ ]:
df

,Category,Id
0,Negativo,3568
1,Neutro,1323
2,Positivo,7976
3,Negativo,2408
4,Negativo,4435
...,...,...
112,Positivo,5279
113,Positivo,6786
114,Negativo,3835
115,Negativo,2859
